In [1]:
from ortools.linear_solver import pywraplp

In [6]:
# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define available resources
resources = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

# Define test intents
# Note: 'have_exprs' and 'want_exprs' are lists of lists representing the 'xor' expressions
# For example, 'have_exprs': [{'a': 3, 'b': 5}, {'a': 2, 'd': 1}] means "(3a or 5b) xor (2a or 1d)"
intents = [
    {'weight': 1.0, 'have': [{'a': 3, 'b': 4, 'c': 5}, {'a': 2}, {'e': 3, 'g': 1}],
     'want': [{'d': 3}, {'f': 1}],
     'resource_weights': {'a': 1.0, 'b': 0.5, 'c': 0.6, 'd': 0.9, 'e': 0.5, 'f': 1.0, 'g': 0.85}
     },
    {'weight': 2.0, 'have': [{'c': 2, 'd': 1}, {'e': 4}, {'f': 2}],
     'want': [{'a': 2, 'b':1}, {'b': 1, 'f': 2}],
     'resource_weights': {'a': 0.7, 'b': 0.2, 'c': 0.9, 'd': 0.9, 'e': 0.85, 'f': 1.0}
    },
]

In [7]:
def resources_in_have(intent):
    unique_resources = set()
    for option in intent['have']:
        unique_resources.update(option.keys())
    return list(unique_resources)

def resources_in_want(intent):
    unique_resources = set()
    for option in intent['want']:
        unique_resources.update(option.keys())
    return list(unique_resources)
resources_in_have = [resources_in_have(intent) for intent in intents]
resources_in_want = [resources_in_want(intent) for intent in intents]

have_choice_vars = []
want_choice_vars = []
have_qty_vars = []
want_qty_vars = []

# Declare variables
for i, intent in enumerate(intents):
    # 1. Boolean variables for each "have" option
    have_bools = [solver.BoolVar(f"have_choice_{i}_{j}") for j in range(len(intent['have']))]
    have_choice_vars.append(have_bools)
    
    # 2. Boolean variables for each "want" option
    want_bools = [solver.BoolVar(f"want_choice_{i}_{k}") for k in range(len(intent['want']))]
    want_choice_vars.append(want_bools)
    
    # 3. Integer variables for each resource traded away
    have_qty = {r: solver.IntVar(0, max_qty, f"have_qty_{i}_{r}") for option in intent['have'] for r, max_qty in option.items()}
    have_qty_vars.append(have_qty)
    
    # 4. Integer variables for each resource received
    want_qty = {r: solver.IntVar(0, qty, f"want_qty_{i}_{r}") for option in intent['want'] for r, qty in option.items()}
    want_qty_vars.append(want_qty)


# Declare constraints
# For each intent, we'll ensure that at most one "have" and one "want" boolean can be true, and they must coincide.
for i in range(len(intents)):
    solver.Add(solver.Sum(have_choice_vars[i]) <= 1)
    solver.Add(solver.Sum(want_choice_vars[i]) <= 1)
    solver.Add(solver.Sum(have_choice_vars[i]) == solver.Sum(want_choice_vars[i]))

# Link "have" and "want" quantities to their choice variables
for i, intent in enumerate(intents):
    for j, have_option in enumerate(intent['have']):
        for r in resources_in_have[i]:
            solver.Add(have_qty_vars[i][r] <= sum(option.get(r, 0) * have_choice_vars[i][j] for j, option in enumerate(intent['have'])))
    
    for k, want_option in enumerate(intent['want']):
        for r in resources_in_want[i]:
            solver.Add(want_qty_vars[i][r] == sum(option.get(r, 0) * want_choice_vars[i][j] for j, option in enumerate(intent['want'])))

# Conservation of each resource
for resource in resources:
    inflow = solver.Sum(have_qty_vars[i].get(resource, 0) for i in range(len(intents)))
    outflow = solver.Sum(want_qty_vars[i].get(resource, 0) for i in range(len(intents)))
    solver.Add(inflow == outflow)

# Declare objective
objective = solver.Objective()

for i, intent in enumerate(intents):
    for r, qty_var in have_qty_vars[i].items():
        coef = intent['weight'] / intent['resource_weights'].get(r, 1)
        objective.SetCoefficient(qty_var, coef)

    for r, qty_var in want_qty_vars[i].items():
        coef = intent['weight'] * intent['resource_weights'].get(r, 1)
        objective.SetCoefficient(qty_var, coef)

objective.SetMaximization()

In [8]:
# Solve the problem
status = solver.Solve()

In [9]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Objective value = {objective.Value()}")
    for i, intent in enumerate(intents):
        if sum(var.solution_value() for var in have_choice_vars[i]) == 1 and sum(var.solution_value() for var in want_choice_vars[i]) == 1:
            traded_haves = []
            traded_wants = []
            for resource, var in have_qty_vars[i].items():
                if var.solution_value() > 0:
                    traded_haves.append(f"{var.solution_value()} {resource}")
            for resource, var in want_qty_vars[i].items():
                if var.solution_value() > 0:
                    traded_wants.append(f"{var.solution_value()} {resource}")
            print(f"Intent {i + 1} trades {' and '.join(traded_haves)} for {' and '.join(traded_wants)}")
else:
    print("The problem does not have an optimal solution.")

Objective value = 10.2
Intent 1 trades 2.0 a and 1.0 b for 1.0 f
Intent 2 trades 1.0 f for 2.0 a and 1.0 b
